# Adversarial Autoencoder: Toronto Face Likelihood Experiments

### Imports

In [10]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from utils import save_weights
import scipy.io
from torch import nn


from torch.utils.data import TensorDataset, DataLoader

import torch
from torch.utils.data import DataLoader, TensorDataset

from src.aae import  AdversarialAutoencoder

from sklearn.model_selection import train_test_split

from likelihood import cross_validate_sigma, estimate_log_likelihood
from utils import compute_mean_std, normalize_data, rescale_to_unit_interval, save_weights

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
data = scipy.io.loadmat('./data/TFD/TFD_48x48.mat')
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'images', 'labs_ex', 'labs_id', 'folds'])

### Preprocessing Pipeline

In [5]:


def configure_tfd(data, device='cuda', batch_size=100):
    images = data['images']
    labels = data['labs_id']

    # Filter unlabeled samples (label == -1)
    mask = labels.flatten() == -1
    if not isinstance(images, torch.Tensor):
        images = torch.tensor(images, dtype=torch.float32)
    images_filtered = images[mask].to(device)

    print(f"Number of unlabeled samples: {images_filtered.shape[0]}")

    print(images_filtered.shape)
    # Flatten
    X = images_filtered.reshape(images_filtered.shape[0], -1)

    # Compute mean and std on GPU
    mean = X.mean(dim=0)
    std = X.std(dim=0)
    X_norm = (X - mean) / std

    # Shuffle and split indices on GPU
    total_samples = X_norm.shape[0]
    perm = torch.randperm(total_samples, device=device)
    train_size = int(0.8 * total_samples)
    train_idx = perm[:train_size]
    test_idx = perm[train_size:]

    X_train = X_norm[train_idx]
    X_test = X_norm[test_idx]

    # Create train DataLoader
    train_dataset = TensorDataset(X_train, torch.zeros(X_train.size(0), dtype=torch.uint8, device=device))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    return train_loader, X_train, X_test, mean, std













## Paper Configuration

In [6]:
INPUT_DIM = 48*48
BATCH_SIZE = 100
AE_HIDDEN = 1000
DC_HIDDEN = 1000
LATENT_DIM = 15
PRIOR_STD = 10.0
recon_loss = nn.MSELoss()
init_recon_lr = 0.01
init_gen_lr = init_disc_lr = 0.1
use_decoder_sigmoid = False

## Training

In [7]:
aae = AdversarialAutoencoder(
    input_dim=INPUT_DIM,
    ae_hidden=AE_HIDDEN,
    dc_hidden=DC_HIDDEN,
    latent_dim=LATENT_DIM,
    recon_loss_fn=recon_loss,
    init_recon_lr=init_recon_lr,
    init_gen_lr=init_gen_lr,
    init_disc_lr=init_disc_lr,
    use_decoder_sigmoid=use_decoder_sigmoid,
    device = device
)

train_loader, X_train, X_test, mean, std = configure_tfd(data=data, device=device, batch_size=100)

Number of unlabeled samples: 98362
torch.Size([98362, 48, 48])


[tensor([[-1.1078, -0.6528, -0.4506,  ..., -0.9035, -0.9044, -0.9440],
        [ 0.5341,  0.5197,  0.5198,  ...,  1.6592,  1.6904,  1.6325],
        [ 0.9585,  1.0222,  1.0050,  ...,  1.0319,  1.2202,  1.2596],
        ...,
        [ 0.3681,  0.1103, -0.0774,  ...,  0.2793, -0.2774, -0.6388],
        [ 0.6633,  0.6686,  0.6131,  ...,  0.8886,  0.7326,  0.2595],
        [ 1.7887,  1.7667,  1.6955,  ...,  0.2434,  0.2276,  0.2595]],
       device='cuda:0')]


In [ ]:
aae.train_mbgd(
    data_loader=train_loader,
    epochs=2000,
    prior_std=PRIOR_STD,
)

Epoch (1/2000)	)Recon Loss: 0.1910	)Disc Loss: 1.3799	)Gen Loss: 0.7210	)


In [11]:
save_weights(encoder=aae.encoder, decoder=aae.decoder, discriminator=aae.discriminator, path_prefix="2000_tfd_aae_weights")

Weights saved to 2000_tfd_aae_weights_*.pth
